In [2]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.2 MB/s eta 0:00:00


In [17]:
from urllib import request
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

from dont_patronize_me import DontPatronizeMe

dpm = DontPatronizeMe('.', '.')

dpm.load_task1()

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [18]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [19]:
import numpy as np
import pandas as pd
import torch
import os
import random

from transformers import DebertaForSequenceClassification, DebertaTokenizer, Trainer, TrainingArguments
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

just making sure that the model generalises well to the dev set

In [21]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

cuda_available = torch.cuda.is_available()
device = torch.device('cuda') if cuda_available else torch.device('cpu')

torch.manual_seed(0)

data=dpm.train_task1_df

trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [22]:
test_rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  test_rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

test_set = pd.DataFrame(test_rows)

In [23]:
test_set

,par_id,community,text,label
0,4046,hopeless,We also know that they can benefit by receivin...,1
1,1279,refugee,Pope Francis washed and kissed the feet of Mus...,1
2,8330,refugee,Many refugees do n't want to be resettled anyw...,1
3,4063,in-need,"""Budding chefs , like """" Fred """" , """" Winston ...",1
4,4089,homeless,"""In a 90-degree view of his constituency , one...",1
...,...,...,...,...
2089,10462,homeless,"The sad spectacle , which occurred on Saturday...",0
2090,10463,refugee,""""""" The Pakistani police came to our house and...",0
2091,10464,disabled,"""When Marie O'Donoghue went looking for a spec...",0
2092,10465,women,"""Sri Lankan norms and culture inhibit women fr...",0


In [24]:
positives_test = test_set[test_set["label"] == 1]
negatives_test = test_set[test_set["label"] == 0]
print(f"positive samples count (test): {len(positives_test)}")
print(f"negative samples count (test): {len(negatives_test)}")

positive samples count (test): 199
negative samples count (test): 1895


In [27]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")
test_encodings = tokenizer(list(test_set["text"]), return_tensors="pt", truncation=True, padding=True).to(device)
test_labels = list(test_set["label"])
test_dataset = Dataset(test_encodings, test_labels)

def compute_metrics(input):
    y_pred = np.argmax(input.predictions, axis=1)
    y_true = input.label_ids
    accuracy = accuracy_score(y_true, y_pred)
    f1score = f1_score(y_true, y_pred)
    return {'accuracy': accuracy, 'f1 score': f1score}

In [28]:
model = DebertaForSequenceClassification.from_pretrained("drive/MyDrive/Deberta/checkpoint-9120").to(device)

In [29]:
training_args = TrainingArguments(
    output_dir="Deberta/",
    learning_rate=1e-5,
    weight_decay=0.05,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1 score",
    greater_is_better=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.evaluate(test_dataset)

<ipython-input-27-109da210e46a>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.7468956708908081,
 'eval_accuracy': 0.9235912129894938,
 'eval_f1 score': 0.5555555555555556,
 'eval_runtime': 27.9544,
 'eval_samples_per_second': 74.908,
 'eval_steps_per_second': 9.372}

Predictions for analysis:

In [30]:
dev_predictions = trainer.predict(test_dataset)
dev_predicted_label_indices = np.argmax(dev_predictions.predictions, axis=1)

par_ids = list(test_set['par_id'])

# predictions_df = pd.DataFrame({
#     'par_id': par_ids,
#     'label': dev_predicted_label_indices
# })

# # Step 4: Save the DataFrame to a CSV file
# predictions_df.to_csv('dev_set_predictions.csv', index=False)

<ipython-input-27-109da210e46a>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [31]:
print(dev_predictions.predictions.shape)
print(dev_predicted_label_indices.shape)

(2094, 2)
(2094,)


In [33]:
with open('dev.txt', 'w') as outf:
  for pred in dev_predicted_label_indices:
    outf.write(str(pred)+'\n')

In [34]:
unseen_data = pd.read_csv("task4_test.tsv",
                       sep="\t",
                       names=['par_id', 'art_id', 'keyword', 'country', 'text', 'label'])

unseen_data['label'] = unseen_data['label'].apply(lambda x: 1)

unseen_data

,par_id,art_id,keyword,country,text,label
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to...",1
1,t_1,@@16397324,women,pk,In most poor households with no education chil...,1
2,t_2,@@16257812,migrant,ca,The real question is not whether immigration i...,1
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population...",1
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken C...",1
...,...,...,...,...,...,...
3827,t_3893,@@20319448,migrant,jm,In a letter dated Thursday to European Commiss...,1
3828,t_3894,@@9990672,poor-families,au,They discovered that poor families with health...,1
3829,t_3895,@@37984,migrant,ca,"She married at 19 , to Milan ( Emil ) Badovina...",1
3830,t_3896,@@9691377,immigrant,us,The United Kingdom is n't going to devolve int...,1


In [35]:
unseen_encodings = tokenizer(list(unseen_data["text"]), return_tensors="pt", truncation=True, padding=True).to(device)
unseen_labels = list(unseen_data["label"])
unseen_dataset = Dataset(unseen_encodings, unseen_labels)

predictions = trainer.predict(unseen_dataset)

<ipython-input-27-109da210e46a>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [36]:
# probabilities = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=-1)

# Get the predicted labels
predicted_label_indices = np.argmax(predictions.predictions, axis=1)

In [37]:
with open('test.txt', 'w') as outf:
  for pred in predicted_label_indices:
    outf.write(str(pred)+'\n')